In [1]:
import cv2 as cv2
import os
import torch
import torchvision
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Type, Dict, Any
import torchvision.models as models
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time
from tqdm import tqdm
import os.path
import imgaug as ia
import os
from threading import Thread
from queue import Empty, Queue
import threading
import random
import accimage
from libs import *
from libs.plot_examples import plot_examples, plot_examples_segmentation
from torch.utils.tensorboard import SummaryWriter
from os.path import join, isfile, isdir
from seg_losses import *


In [2]:
resize = 512
tb_period = 8

In [3]:
run_prefix = 'Unet_CoordConv_mish_resize512'

In [4]:
# region logs_basepath
existing_logs_directories = [d for d in find_directories('./logs', '%s_run*' % run_prefix, maxdepth=2)]
prev_runs = [os.path.basename(os.path.split(d)[0]) for d in existing_logs_directories]
prev_runs = [int(s.replace('%s_run' % run_prefix, '')) for s in prev_runs]
if len(prev_runs) > 0:
    curr_run = np.max(prev_runs) + 1
else:
    curr_run = 1
curr_run = '%s_run%04d' % (run_prefix, curr_run)
logs_basepath = os.path.join('./logs', curr_run)
tb_basepath = os.path.join('./TBoard', curr_run)
EnsureDirectoryExists(logs_basepath)

checkpoints_basepath = os.path.join('./movdel_checkpoints', curr_run)
EnsureDirectoryExists(checkpoints_basepath)
# endregion


In [5]:
tb_writer = SummaryWriter(log_dir=tb_basepath)

device1 = torch . device ( "cuda:0" if torch . cuda . is_available () else "cpu" )
device2 = torch.device('cpu')

In [6]:
model = UnetCoordConv(activation = Mish)
for param in model.parameters():
    param.requires_grad = True
model = nn.DataParallel(model)
model = model.to(device1)

In [7]:
model

DataParallel(
  (module): UnetCoordConv(
    (conv_block1): Sequential(
      (0): CoordConv2d(
        (conv): Conv2d(4, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Mish()
      (3): CoordConv2d(
        (conv): Conv2d(129, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Mish()
    )
    (conv_block2): Sequential(
      (0): CoordConv2d(
        (conv): Conv2d(129, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Mish()
      (3): CoordConv2d(
        (conv): Conv2d(257, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
      (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (

In [8]:
#region threading
class thread_killer(object):
    """Boolean object for signaling a worker thread to terminate"""

    def __init__(self):
        self.to_kill = False

    def __call__(self):
        return self.to_kill

    def set_tokill(self, tokill):
        self.to_kill = tokill


def threaded_batches_feeder(tokill, batches_queue, dataset_generator):
    while tokill() == False:
        for img, landmarks in dataset_generator:
            batches_queue.put((img, landmarks), block=True)
            if tokill() == True:
                return


def threaded_cuda_batches(tokill, cuda_batches_queue, batches_queue):
    while tokill() == False:
        (img, landmarks) = batches_queue.get(block=True)
        img = torch.from_numpy(img)
        landmarks = torch.from_numpy(landmarks)
        img = Variable(img.float()).to(device1)
        landmarks = Variable(landmarks.float()).to(device1)
        cuda_batches_queue.put((img, landmarks), block=True)

        if tokill() == True:
            return
#endregion

In [9]:
batch_size = 4
sun_dataset_train = SunDiskDataset(csv_file='sun_disk_pos_database01train.csv',
                                   root_dir='./images',
                                   resize=resize,
                                   batch_size=batch_size)
sun_dataset_test = SunDiskDataset(csv_file='sun_disk_pos_database01test.csv',
                                  root_dir='./images',
                                  resize=resize,
                                  batch_size=batch_size,
                                  augment=False)


In [10]:
def train_single_epoch(model: torch.nn.Module,
                       optimizer: torch.optim.Optimizer,
                       loss_function: torch.nn.Module,
                       cuda_batches_queue: Queue,
                       Per_Step_Epoch: int,
                       current_epoch: int):
    model.train()
    loss_values = []
    loss_tb = []
    pbar = tqdm(total=Per_Step_Epoch)
    for batch_idx in range(int(Per_Step_Epoch)):  # тут продумать
        data_image, target = cuda_batches_queue.get(block=True)

        ###### DEBUG plots
        if batch_idx == 0:
            plot_examples_segmentation(data_image, target, file_output = os.path.join(logs_basepath, 'train_input_ep%04d.png' % current_epoch))
        ###### DEBUG plots
        target = Variable(target)
        optimizer.zero_grad()  # обнулили\перезапустии градиенты для обратного распространения
        data_out = model(data_image)  # применили модель к данным
        loss = loss_function(data_out, target)  # применили фуннкцию потерь
        loss_values.append(loss.item())

        loss_tb.append(loss.item())
        tb_writer.add_scalar('train_loss', np.mean(loss_tb), current_epoch*Per_Step_Epoch + batch_idx)
        loss_tb=[]

        loss.backward()  # пошли по графу нейросетки обратно
        optimizer.step()  # выполняем наш градиентный спуск по вычисленным шагам в предыдущей строчке
        pbar.update(1)
        pbar.set_postfix({'loss': loss.item()})
    pbar.close()
    #MK не забывай закрывать pbar

    return np.mean(loss_values)


In [11]:
def validate_single_epoch(model: torch.nn.Module,
                          loss_function: torch.nn.Module,
                          cuda_batches_queue: Queue,
                          Per_Step_Epoch: int,
                          current_epoch: int):
    model.eval()

    loss_values = []

    pbar = tqdm(total=Per_Step_Epoch)
    for batch_idx in range(int(Per_Step_Epoch)):  # тут продумать
        data_image, target = cuda_batches_queue.get(block=True)
        data_out = model(data_image)

        if batch_idx == 0:
            plot_examples_segmentation(data_image, data_out, file_output = os.path.join(logs_basepath, 'val_results_ep%04d.png' % current_epoch))
#         print(5)

        loss = loss_function(data_out, target)
        loss_values.append(loss.item())
        pbar.update(1)
        pbar.set_postfix({'loss': loss.item()})
    pbar.close()
    # MK не забывай закрывать pbar

    return np.mean(loss_values)


In [12]:
def train_model(model: torch.nn.Module,
                train_dataset: Dataset,
                val_dataset: Dataset,
                max_epochs=480):

    loss_function = BinaryFocalLoss(alpha=1, gamma=4)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    lr_scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=32, T_mult=2, eta_min=1e-8, lr_decay = 0.8)

    #region data preprocessing threads starting
    batches_queue_length = 8
    preprocess_workers = 1

    train_batches_queue = Queue(maxsize=batches_queue_length)
    train_cuda_batches_queue = Queue(maxsize=4)
    train_thread_killer = thread_killer()
    train_thread_killer.set_tokill(False)

    for _ in range(preprocess_workers):
        thr = Thread(target=threaded_batches_feeder, args=(train_thread_killer, train_batches_queue, train_dataset))
        thr.start()

    train_cuda_transfers_thread_killer = thread_killer()
    train_cuda_transfers_thread_killer.set_tokill(False)
    train_cudathread = Thread(target=threaded_cuda_batches, args=(train_cuda_transfers_thread_killer, train_cuda_batches_queue, train_batches_queue))
    train_cudathread.start()

    test_batches_queue = Queue(maxsize=batches_queue_length)
    test_cuda_batches_queue = Queue(maxsize=4)
    test_thread_killer = thread_killer()
    test_thread_killer.set_tokill(False)

    for _ in range(preprocess_workers):
        thr = Thread(target=threaded_batches_feeder, args=(train_thread_killer, test_batches_queue, sun_dataset_test))
        thr.start()

    test_cuda_transfers_thread_killer = thread_killer()
    test_cuda_transfers_thread_killer.set_tokill(False)
    test_cudathread = Thread(target=threaded_cuda_batches,
                             args=(test_cuda_transfers_thread_killer, test_cuda_batches_queue,
                                   test_batches_queue))
    test_cudathread.start()
    #endregion

    Steps_Per_Epoch_Train = 64
    Steps_Per_Epoch_Test = len(val_dataset) // batch_size + 1
    best_val_loss = None

    for epoch in range(max_epochs):

        print(f'Epoch {epoch} / {max_epochs}')
        train_loss = train_single_epoch(model, optimizer, loss_function, train_cuda_batches_queue, Steps_Per_Epoch_Train, current_epoch=epoch)

        tb_writer.add_scalar('train_loss', train_loss, epoch)

        val_loss = validate_single_epoch(model, loss_function, test_cuda_batches_queue, Steps_Per_Epoch_Test, current_epoch=epoch)
        tb_writer.add_scalar('val_loss', val_loss, epoch)

        print(f'Validation loss: {val_loss}')
        if best_val_loss is None or best_val_loss > val_loss:
            print(f'Best model yet, saving')
            best_val_loss = val_loss
            #best_epoch = epoch
            torch.save(model, 'saved_models/Unet.pth') 

        lr_scheduler.step()

        torch.save(model.module.state_dict(), os.path.join(checkpoints_basepath, 'model_ep%04d.pt' % epoch))

    #region stopping datapreprocessing threads
    test_thread_killer.set_tokill(True)  # убиваю потокои, так же убить валидационные
    train_thread_killer.set_tokill(True)  # убиваю потокои, так же убить валидационные
    test_cuda_transfers_thread_killer.set_tokill(True)
    train_cuda_transfers_thread_killer.set_tokill(True)
    for _ in range(preprocess_workers):
        try:
            # Enforcing thread shutdown
            test_batches_queue.get(block=True, timeout=1)
            test_cuda_batches_queue.get(block=True, timeout=1)
            train_batches_queue.get(block=True, timeout=1)
            train_cuda_batches_queue.get(block=True, timeout=1)
        except Empty:
            pass
    #endregion



In [13]:
train_model(model,
            train_dataset = sun_dataset_train,
            val_dataset =  sun_dataset_test,
            max_epochs=480)

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 0 / 480


/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:3122: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
100%|██████████| 134/134 [00:35<00:00,  4.81it/s, loss=0.894]


Validation loss: 0.8898255673807058
Best model yet, saving


  0%|          | 0/64 [00:00<?, ?it/s]

...

Validation loss: 0.6931473016738892
Epoch 478 / 480


100%|██████████| 134/134 [00:28<00:00,  4.87it/s, loss=0.693]


Validation loss: 0.6931473016738892
Epoch 479 / 480


100%|██████████| 134/134 [00:28<00:00,  4.87it/s, loss=0.693]

Validation loss: 0.6931473016738892


In [16]:
6+2

8